# Problem Statement :
**In this project we have to build a system which summarizes the customer reviews of a particular product into a bunch of keywords, so that when a customer goes to a product page, he/she doesn’t have to read long reviews. Instead he/she can make up his/her mind based on the product average rating and summarized keywords of the review.**

In [1]:
import pandas as pd
import string
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_json("C:/Users/Arjun Rao/Desktop/DS/NLP/review summary/Cell_Phones_and_Accessories.json")

In [3]:
df

,Rating,Rev_verify,Review_Date,IC,Prod_meta,Reviewer_Name,Review,Rev_summ,Review_timestamp,Useful,Prod_img
0,5,True,"09 1, 2015",B009XD5TPQ,None,Sunny Zoeller,Bought it for my husband. He's very happy with it,He's very happy with,1441065600,None,None
1,5,True,"01 9, 2016",B016MF3P3K,None,Denise Lesley,Great screen protector. Doesn't even seem as ...,Five Stars,1452297600,None,None
2,5,True,"04 21, 2013",B008DC8N5G,None,Emir,Saved me lots of money! it's not gorilla glass...,As long as you know how to put it on!,1366502400,None,None
3,3,True,"02 27, 2013",B0089CH3TM,{'Color:': ' Green'},Alyse,"The material and fit is very nice, but the col...",Good case overall,1361923200,3,None
4,4,True,"12 19, 2013",B00AKZWGAC,None,TechGuy,This last me about 3 days till i have to charg...,Awesome Battery,1387411200,None,None
...,...,...,...,...,...,...,...,...,...,...,...
760445,4,False,"07 12, 2014",B00C3V9M8A,None,momahjoub,Very good,Four Stars,1405123200,None,None
760446,5,False,"07 13, 2016",B0178BYS24,None,Cindy,My name is Cynthia Beard and I believe that th...,... believe that the Samsung Galaxy car mount ...,1468368000,None,None
760447,4,True,"07 23, 2015",B009KY47CE,None,zzrnam11,This iphone case is very durable and long last...,I LOVE THIS,1437609600,None,None
760448,5,True,"12 14, 2015",B00X60AYDY,{'Style:': ' 6-in-1 Silver'},ACER,great,Five Stars,1450051200,None,None


**About the dataset**

This .json file contains review information under following columns:
    
. IC – Item Code of the product, e.g. B016MF3P3K

. Reviewer_Name - Name of the reviewer

. Useful- Number of useful votes (upvotes) of the review

. Prod_meta- a dictionary of the product metadata. It contains only additional information about the product, if any available.

. Review- text of the review

. Rating- rating given to the product by the reviewer.

. Rev_summ- summary of the review

. Review_timestamp- time when the review has been posted (unix time format)

. Review_Date- Date when the review has been posted

. Prod_img- images that users post after they have received the product

. Rev_verify- Flag to represent whether the review has been verified or not. (True/False)

In [4]:
# Check for null data
df.isnull().sum()

Rating                   0
Rev_verify               0
Review_Date              0
IC                       0
Prod_meta           352624
Reviewer_Name           91
Review                 530
Rev_summ               355
Review_timestamp         0
Useful              698250
Prod_img            742256
dtype: int64

**As we know we have Rating,Review & IC as crucial parameters so we consider to remove the rows have null values for these parameters.**

In [5]:
#Checking total number of unique item codes we have before removing null rows
df.IC.nunique()

48134

In [6]:
#Removing rows with null Review 
df.dropna(subset=['Review'],inplace=True)

In [7]:
df.isnull().sum()

Rating                   0
Rev_verify               0
Review_Date              0
IC                       0
Prod_meta           352384
Reviewer_Name           90
Review                   0
Rev_summ               318
Review_timestamp         0
Useful              697743
Prod_img            741786
dtype: int64

In [8]:
# Checking unique values of Item code we have with value counts
df.IC.value_counts()

B00VH88CJ0    1125
B005NF5NTK    1106
B019PV2I3G    1033
B00X5RV14Y    1018
B00P7N0320    1011
              ... 
B0159O6MOE       1
B018E6OZVQ       1
B00ID9F640       1
B00P21OYAE       1
B007DBWJ8K       1
Name: IC, Length: 48133, dtype: int64

**we have left with total 48133 unique product codes after removing null values**

# Data Preprocessing and Cleansing 

In [9]:
#Let's arrange our dataset by making a pivot table Item code wise 
table= pd.pivot_table(df, index=['IC','Review'])
table

Rating  \
IC         Review                                                       
7508492919 Another great product that my daughter she use ...     5.0   
           Beautiful quality and outstanding product! Ever...     5.0   
           Can not argue with the price or appearance. Loo...     3.0   
           Cute case. Jewels do come off but what do you e...     4.0   
           For the price i received a pretty great case! I...     4.0   
...                                                               ...   
B01HJH9IN6 Just received it and it's VERY thin. No need to...     1.0   
           NOT WORTH FOR THE PRICE. CAME BROKEN                   1.0   
           Previously had a metallic bumper purchased here...     2.0   
           This case is nice but definitely for looks rath...     3.0   
           Totally love these cases. They actually, with t...     5.0   

                                                               Rev_verify  \
IC         Review                                                           
7508492919 Another great product that my daughter she use ...         1.0   
           Beautiful quality and outstanding product! Ever...         1.0   
           Can not argue with the price or appearance. Loo...         1.0   
           Cute case. Jewels do come off but what do you e...         1.0   
           For the price i received a pretty great case! I...         1.0   
...                                                                   ...   
B01HJH9IN6 Just received it and it's VERY thin. No need to...         1.0   
           NOT WORTH FOR THE PRICE. CAME BROKEN                       1.0   
           Previously had a metallic bumper purchased here...         1.0   
           This case is nice but definitely for looks rath...         1.0   
           Totally love these cases. They actually, with t...         1.0   

                                                               Review_timestamp  
IC         Review                                                                
7508492919 Another great product that my daughter she use ...      1.382486e+09  
           Beautiful quality and outstanding product! Ever...      1.378771e+09  
           Can not argue with the price or appearance. Loo...      1.373760e+09  
           Cute case. Jewels do come off but what do you e...      1.355270e+09  
           For the price i received a pretty great case! I...      1.359418e+09  
...                                                                         ...  
B01HJH9IN6 Just received it and it's VERY thin. No need to...      1.472688e+09  
           NOT WORTH FOR THE PRICE. CAME BROKEN                    1.479168e+09  
           Previously had a metallic bumper purchased here...      1.479859e+09  
           This case is nice but definitely for looks rath...      1.471478e+09  
           Totally love these cases. They actually, with t...      1.468886e+09  

[752374 rows x 3 columns]

In [10]:
#Converting dataset to required format

# Lets combine all reviews for particular porducts using groupby function 
group1= pd.DataFrame(df.groupby(['IC'],as_index=True)['Review'].apply(lambda tags: ','.join(tags)))
group2= df.groupby(['IC'],as_index=False).agg({'Rating':['min','max','mean']})
grp1=group1.reset_index()
pd.options.display.float_format = '{:.1f}'.format #Formatting data to 1 decimal place(using for mean Ratings )

result=pd.merge(grp1,group2,on="IC")
result.head()

,IC,Review,"(Rating, min)","(Rating, max)","(Rating, mean)"
0,7508492919,It is such a good case for a low price. I have...,1,5,4.3
1,7532385086,I was very excited when I first got this case....,1,5,3.3
2,7887421268,"Good item,Received this in about a weeks time....",1,5,3.5
3,8199900164,"thank you.,Works some of the time. The charge...",2,5,3.8
4,8288853439,Great ! So far.\nI had this charger before and...,1,5,3.8


In [11]:
import nltk
from nltk.corpus import  stopwords

# Defining stopwords , we have dedicated list for stopwords other than that of NLTK stopwords
file=open("C:/Users/Arjun Rao/Desktop/keys.txt")
text = file.read() 
stp_wrds=text.split()
stop_words = set(stopwords.words('english')+ stp_wrds)

In [12]:
#Removing numbers,email,website and other useless data content form our reviews to clean the data
comp_df = result['Review'].str.replace(r'[^a-zA-Z]',' ')
comp_df=comp_df.str.replace(r'[^\w\d\s]', ' ').str.lower()
comp_df=comp_df.astype(str)
comp_df.head()

0    it is such a good case for a low price  i have...
1    i was very excited when i first got this case ...
2    good item received this in about a weeks time ...
3    thank you  works some of the time   the charge...
4    great   so far  i had this charger before and ...
Name: Review, dtype: object

# Training our model to extract keywords for unique products 

**We use NTLK-RAKE algorithm to extract keywords from our text/reviews**


**RAKE short for Rapid Automatic Keyword Extraction algorithm, is a domain independent keyword extraction algorithm which tries to determine key phrases in a body of text by analyzing the frequency of word appearance and its co-occurance with other words in the text.**

**After identifying all the keywords from the text data, a graph of word co-occurrence is generated which calculates the score for each  keyword and defined as the member word score. With the help of this graph, we evaluate several metrics for calculating word scores, based on the degree and frequency of the vertices in the graph.**



In [ ]:
#!pip install rake-nltk

In [15]:
import nltk
from rake_nltk import Rake,Metric

In [16]:
# Rake uses stopwords for english from NLTK, and all puntuation characters + we have assigned our own set of keywords

# We have assigned minimum and maximum length of words for keywords and metric as word frequency to get out best of our model

r=Rake(stopwords=stop_words,min_length=1, max_length=3,ranking_metric=Metric.WORD_FREQUENCY) 

def rake_implement(x,r):
    r.extract_keywords_from_text(x)
    return r.get_ranked_phrases()[0:20] # We are taking first 20 keywords with top word score

result['key_words'] =comp_df.apply(lambda x: rake_implement(x,r))
result.head()

,IC,Review,"(Rating, min)","(Rating, max)","(Rating, mean)",key_words
0,7508492919,It is such a good case for a low price. I have...,1,5,4.3,"[pretty great case, beautiful phone case, nice..."
1,7532385086,I was very excited when I first got this case....,1,5,3.3,"[phone received, phone perfectly, started noti..."
2,7887421268,"Good item,Received this in about a weeks time....",1,5,3.5,"[fit phone case, nice case, best protection ca..."
3,8199900164,"thank you.,Works some of the time. The charge...",2,5,3.8,"[phone beeps, doesnt work, work, stopped worki..."
4,8288853439,Great ! So far.\nI had this charger before and...,1,5,3.8,"[blasted blackberry charger, samsung battery c..."


In [17]:
pd.set_option('display.max_colwidth', -1)
result['key_words'].head()

0    [pretty great case, beautiful phone case, nice phone case, phone case, cute case, pretty phone kind, iphone case, pretty good quality, good case, girly case, grand daughters phone, easily soooo cute, cutest case, soo super pretty, pretty nicely, phone cover, cute love, case, super cute, phone face]                                     
1    [phone received, phone perfectly, started noticing stuff, pretty case, phone, highly recommend, complete refund, soft rubberized, slight issue, rubber peels, refund, recommend, received, order covers, honest company, excellent buy, case, white, wanting, wanted]                                                                           
2    [fit phone case, nice case, best protection case, case started, missing spots guess, good item received, case, nice feel, black leopard spots, product recommended buy, nice, missing paint, good deal, weeks time, stay clipped, received, protection, pink purple, phone, outter box]                                

**We can see that we have meaningful keywords extracted from our reviews , we can simply go though these keywords to get idea about the product**

**Final data we have**

In [18]:
result.head()

IC  \
0  7508492919   
1  7532385086   
2  7887421268   
3  8199900164   
4  8288853439   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

**converting and saving final dataframe to .json file**

In [19]:

result.to_json(path_or_buf="C:/Users/Arjun Rao/Desktop/DS/NLP/review summary/Cell_Phones_and_Accessories_keywords.json",orient="split")